In [1]:
matrix?

Signature:      matrix(*args, **kwds)
Call signature: matrix(*args, **kwargs)
Type:           cython_function_or_method
String form:    <cyfunction matrix at 0x148506a80>
File:           /private/var/tmp/sage-10.6-current/local/var/lib/sage/venv-python3.12.5/lib/python3.12/site-packages/sage/matrix/constructor.pyx
Docstring:     
   Create a matrix.

   This implements the "matrix" constructor:

      sage: matrix([[1,2],[3,4]])
      [1 2]
      [3 4]

   It also contains methods to create special types of matrices, see
   "matrix.[tab]" for more options. For example:

      sage: matrix.identity(2)
      [1 0]
      [0 1]

   INPUT:

   The "matrix()" command takes the entries of a matrix, optionally
   preceded by a ring and the dimensions of the matrix, and returns a
   matrix.

   The entries of a matrix can be specified as a flat list of
   elements, a list of lists (i.e., a list of rows), a list of Sage
   vectors, a callable object, or a dictionary having positions as
   keys a

In [1]:
def structured_matrix(n, d_list):
    """
    Constructs the n x n matrix A with:
    - A[i,j] = d_{i-j} for i >= j (lower diagonals, including main)
    - A[i,i+1] = i+1 for i = 0 to n-2 (first superdiagonal)
    - all other entries zero

    Input:
        n      : integer >= 1
        d_list : list of length n, with values [d_0, d_1, ..., d_{n-1}]

    Output:
        A : n x n matrix over the parent of d_list[0]
    """
    if len(d_list) != n:
        raise ValueError("d_list must have length n")
    
    # Determine base ring from d_list
    R = parent(d_list[0])
    A = Matrix(R, n, n, 0)  # initialize to zero matrix

    for i in range(n):
        for j in range(n):
            if i >= j:
                A[i, j] = d_list[i - j]  # subdiagonals (including main)
            elif j == i + 1:
                A[i, j] = i + 1          # superdiagonal entries 1, 2, ..., n-1

    return A


In [2]:
d_list=[5,7]
n=2
structured_matrix(n, d_list)

[5 1]
[7 5]

In [ ]:
def structured_determinant(n, d_list):
    """
    Compute det(A_n) for the matrix defined by structured_matrix(n, d_list)
    using a recurrence:
    
        D_n = d_0 D_{n-1} - (n-1)d_1 D_{n-2} + (n-2)d_2 D_{n-3} - ...
    
    Input:
        n      : integer >= 1
        d_list : list of length >= n, values [d_0, d_1, ..., d_{n-1}]
                 must be elements of a symbolic ring or field

    Output:
        D_n : symbolic expression for determinant
    """
    if len(d_list) < n:
        raise ValueError("d_list must be at least length n")
    
    # Use SR for symbolic work
    D = [SR(1)]  # D_0 = 1
    
    for k in range(1, n + 1):
        term = d_list[0] * D[k - 1]
        for j in range(1, k):
            sign = (-1)**j
            coeff = (k - j)
            term += sign * coeff * d_list[j] * D[k - j - 1]
        D.append(term)

    return D[n]


In [ ]:
def structured_matrix(n, d_list):
    """
    Construct the n x n matrix A with:
      - entries on and below the main diagonal given by d_list (d_0 on main diagonal,
        d_1 on first subdiagonal, ..., d_{n-1} on last subdiagonal),
      - entries on the first superdiagonal equal to 1, 2, ..., n-1,
      - all other entries zero.

    INPUT:
      - n : positive integer
      - d_list : list of length n with diagonal values [d_0, d_1, ..., d_{n-1}]
    
    OUTPUT:
      - n x n matrix over parent ring of d_list elements
    """
    if len(d_list) != n:
        raise ValueError("d_list length must be equal to n")
    
    R = parent(d_list[0])
    A = Matrix(R, n, n, 0)  # initialize zero matrix
    
    for i in range(n):
        for j in range(n):
            if i >= j:
                A[i, j] = d_list[i - j]  # on and below main diagonal
            elif j == i + 1:
                A[i, j] = i + 1          # superdiagonal entries: 1,2,...,n-1
            else:
                A[i, j] = 0              # zeros elsewhere
    
    return A


In [1]:
# Importing necessary library
A = Matrix([[4, -2], [1, 1]])

# Find eigenvalues of matrix A
eigenvalues = A.eigenvalues()
print("Eigenvalues:", eigenvalues)


Eigenvalues: [3, 2]


In [5]:
# Define the matrix
A = Matrix([[2, 1], 
            [1, 2]])

# Compute the eigenvectors
eigenvectors = A.eigenvectors_right()

# Display the eigenvectors
for eigval, eigvecs, multiplicity in eigenvectors:
    print(f"Eigenvalue: {eigval}")
    print(f"Multiplicity: {multiplicity}")
    print("Eigenvectors:")
    for v in eigvecs:
        print(v)
    print()

Eigenvalue: 3
Multiplicity: 1
Eigenvectors:
(1, 1)

Eigenvalue: 1
Multiplicity: 1
Eigenvectors:
(1, -1)



In [9]:
Rx.<x>= PolynomialRing(QQ)
Pm = PolynomialRing(QQ, 'm')
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)
def H_matrix(h_list):
    n = len(h_list)
    H = Matrix(QQ, n, n)

    # Column 0 (first column): i*h_i for i from 1 to n
    for i in range(n):
        H[i, 0] = (i + 1) * h_list[i]

    # Column 1 and onwards: each pushes down previous column
    for col in range(1, n):
        for row in range(n):
            if row == 0:
                H[row, col] = 0
            else:
                H[row, col] = H[row - 1, col - 1]

    return H

In [4]:
h_list=[10,10,10]
H_matrix(h_list)

[10  0  0]
[20 10  0]
[30 20 10]

In [12]:
h_list=[]
for n in [1,2,3]:
    h_list+=[ramanujanTau(n+1)]
H_matrix(h_list)

[  -24     0     0]
[  504   -24     0]
[-4416   504   -24]

In [92]:
23*64

1472

In [2]:
import pickle 
Rx.<x>= PolynomialRing(QQ)
x = var('x')
def H_matrix(h_list):
    n = len(h_list)
    H = Matrix(QQ, n, n)

    # Column 0 (first column): i*h_i for i from 1 to n
    for i in range(n):
        H[i, 0] = (i + 1) * h_list[i]

    # Column 1 and onwards: each pushes down previous column
    for col in range(1, n):
        for row in range(n):
            if row == 0:
                H[row, col] = 0
            else:
                H[row, col] = H[row - 1, col - 1]

    return H

def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)

charpolys=[]

for N in range(1, 5):
    
    inputs = [1 for j in range(1, N + 1)]
    hm = H_matrix(inputs)
    print(hm)
    print()

[1]

[1 0]
[2 1]

[1 0 0]
[2 1 0]
[3 2 1]

[1 0 0 0]
[2 1 0 0]
[3 2 1 0]
[4 3 2 1]



In [10]:
Rx.<x> = PolynomialRing(QQ)
from collections import deque

def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)

def S(n):
    return 24 * (-1)^n * sigma(n)

def T(n):
    return ZZ(ramanujanTau(n+1))

def c1_operator(lst):
    return [(k + 1) * lst[k] for k in range(len(lst))]

def push_down(j, N, lst):
    d = [1] + lst
    for _ in range(2, j + 1):
        d = [0] + d
    return d[:N]

# MAIN
N = 4
lst = [T(j) for j in range(1, N + 1)]

# Build the matrix columns
column_list = [vector(ZZ, c1_operator(lst))]
for j in range(1, N):
    pushed = push_down(j, N, lst)
    column_list.append(vector(ZZ, pushed))

# Construct the Sage integer matrix
mat = Matrix(ZZ, column_list).transpose()

# Compute exact determinant
determinant = mat.det()
expected = 24 * sigma(N) * (-1)^N
test = determinant - expected

# Output
print("Matrix:")
print(mat)
print("Determinant:", determinant)
print("Test (should be 0):", test)


Matrix:
[  -24     1     0     0]
[  504   -24     1     0]
[-4416   252   -24     1]
[19320 -1472   252   -24]
Determinant: 168
Test (should be 0): 0


In [13]:
Rx.<x> = PolynomialRing(QQ)
from collections import deque
import pickle
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)

def S(n):
    return 24 * (-1)^n * sigma(n)

def T(n):
    return ZZ(ramanujanTau(n+1))

def c1_operator(lst):
    return [(k + 1) * lst[k] for k in range(len(lst))]

def push_down(j, N, lst):
    d = [1] + lst
    for _ in range(2, j + 1):
        d = [0] + d
    return d[:N]

# MAIN
tests=[]
charpolys=[]
for N in range(1,101):
    lst = [T(j) for j in range(1, N + 1)]

# Build the matrix columns
    column_list = [vector(ZZ, c1_operator(lst))]
    for j in range(1, N):
        pushed = push_down(j, N, lst)
        column_list.append(vector(ZZ, pushed))

# Construct the Sage integer matrix
    mat = Matrix(ZZ, column_list).transpose()

# Compute exact determinant
    determinant = mat.det()
    p = mat.charpoly(x)#.change_ring(SR)
    charpolys+=[(N,p)]
    expected = 24 * sigma(N) * (-1)^N
    test = determinant - expected
    tests+=[test]

# Output
print(tests)

wfile = open('/Users/barrybrent/data/run6aug25no1.txt','wb') 
t = pickle.dumps(str(charpolys))
wfile.write(t)
wfile.close() 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)
Rx= PolynomialRing(QQ, 'x')
Pm = PolynomialRing(QQ, 'm')
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def rdr(p,n):
    return QQ.valuation(p)(n)
def sig(j):
    return 24*sigma(j)
import pickle
rfile = open('/Users/barrybrent/data/run6aug25no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
cffx0=[]
data=[]
for k in range(1,len(s)):
        N = s[k][0]
        n=0
        poly=Rx(s[k][1])
        epc=extendedPolynomialCoefficient(n,poly)
        data+=[epc/24]
            
print(data)

[3, 4, 7, 6, 12, 8, 15, 13, 18, 12, 28, 14, 24, 24, 31, 18, 39, 20, 42, 32, 36, 24, 60, 31, 42, 40, 56, 30, 72, 32, 63, 48, 54, 48, 91, 38, 60, 56, 90, 42, 96, 44, 84, 78, 72, 48, 124, 57, 93, 72, 98, 54, 120, 72, 120, 80, 90, 60, 168, 62, 96, 104, 127, 84, 144, 68, 126, 96, 144, 72, 195, 74, 114, 124, 140, 96, 168, 80, 186, 121, 126, 84, 224, 108, 132, 120, 180, 90, 234, 112, 168, 128, 144, 120, 252, 98, 171, 156, 217]


In [23]:
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)
Rx= PolynomialRing(QQ, 'x')
Pm = PolynomialRing(QQ, 'm')
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def rdr(p,n):
    return QQ.valuation(p)(n)
def sig(j):
    return 24*sigma(j)
import pickle
rfile = open('/Users/barrybrent/data/run6aug25no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
cffx0=[]
data=[]
for k in range(1,len(s)):
        N = s[k][0]
        n=0
        poly=Rx(s[k][1])
        epc=extendedPolynomialCoefficient(n,poly)
        data+=[epc/24-sigma(N)]
            
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [42]:
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)
Rx= PolynomialRing(QQ, 'x')
Pm = PolynomialRing(QQ, 'm')
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def rdr(p,n):
    return QQ.valuation(p)(n)
def sig(j):
    return 24*sigma(j)
import pickle
rfile = open('/Users/barrybrent/data/run6aug25no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
cffx0=[]
data=[]
two_rdrs=[]
three_rdrs=[]
for k in range(1,len(s)):
        N = s[k][0]
        n=1
        poly=Rx(s[k][1])
        epc=extendedPolynomialCoefficient(n,poly)
        den=factorial(n)*binomial(N,n)
        data+=[epc/den]
print(data)          
#A006922 Expansion of 1/eta(q)^24; Fourier coefficients of T_{14}
# which is (per ChatGPT) & probably my exp math paper) 
# the Hauptmoduln for Gamma_0(14^+)
# also see A010922 Pisot sequence T(14,23), a(n)=[ a(n-1)^2/a(n-2) ].

[24, 324, 3200, 25650, 176256, 1073720, 5930496, 30178575, 143184000, 639249300, 2705114880, 10914317934, 42189811200, 156883829400, 563116739584, 1956790259235, 6599620022400, 21651325216200, 69228721526400, 216108718571250, 659641645039360, 1971466420726656, 5776331152550400, 16610409114771900, 46925988716146176, 130362155499200220, 356418628326241024, 959788304511313500, 2547447689037081600, 6668597583531616856, 17227666361525437440, 43946595512833354821, 110753578062185091200, 275889636433651636800, 679603117953171550464, 1656159528253893300680, 3994373142513720019584, 9537992210458653910200, 22556911735643814336000, 52851854116498243371768, 122725297901736598060800, 282506903283485314589800, 644860824976888592486400, 1460021679052070827818150, 3279574124861933907622400, 7310437360199294416934040, 16174647177339633952121856, 35529150168048315816004075, 77496468553699048793894400, 167884450803343339733543652, 361285884535944808280618880, 772467683080630774831410348, 1641239217764259

In [ ]:
# LaTeX: The Meaning of the “+” in $\Gamma_0(N)^+$

The “+” in the notation $\Gamma_0(N)^+$ indicates an extension of the modular group $\Gamma_0(N)$ by including certain **Atkin–Lehner involutions**, most notably the **Fricke involution** $w_N$. This forms a larger group often called the **Fricke group** at level $N$.

---

## 1. The group $\Gamma_0(N)$

This is the congruence subgroup of $\mathrm{SL}_2(\mathbb{Z})$ defined as:

$$
\Gamma_0(N) = \left\{ 
\begin{pmatrix} a & b \\ c & d \end{pmatrix} \in \mathrm{SL}_2(\mathbb{Z}) \;\middle|\; c \equiv 0 \pmod{N}
\right\}.
$$

---

## 2. The Fricke involution $w_N$

The **Fricke involution** at level $N$ is:

$$
w_N = \begin{pmatrix} 0 & -1/\sqrt{N} \\ \sqrt{N} & 0 \end{pmatrix}.
$$

Although $w_N \notin \mathrm{SL}_2(\mathbb{Z})$, it normalizes $\Gamma_0(N)$ and acts on modular forms. It maps $\tau \mapsto -1/(N\tau)$ and exchanges the cusps $\infty$ and $0$.

---

## 3. The group $\Gamma_0(N)^+$

This group is defined by adjoining $w_N$ to $\Gamma_0(N)$:

$$
\Gamma_0(N)^+ = \langle \Gamma_0(N), w_N \rangle
$$

It is a finite-index subgroup of $\mathrm{PSL}_2(\mathbb{R})$ and is commonly referred to as the **Fricke modular group**.

---

## Why is this important?

- $\Gamma_0(N)^+$ identifies more points on the modular curve than $\Gamma_0(N)$.
- For many $N$, the modular curve $X_0(N)^+ = \Gamma_0(N)^+ \backslash \mathbb{H}^*$ has **genus zero**, even when $X_0(N)$ does not.
- When this happens, a **Hauptmodul** (like $j(\tau)$) exists, usually denoted $T_N(q)$.

---

## Example

The modular function

$$
T_{14}(q) = \frac{1}{q} + 24 + 324q + 3200q^2 + 25650q^3 + \cdots
$$

is the Hauptmodul for $\Gamma_0(14)^+$. It is a modular function of weight 0, with a simple pole at $\infty$ and rational coefficients.

---

## Summary

- The “+” in $\Gamma_0(N)^+$ indicates inclusion of Atkin–Lehner involutions.
- The group $\Gamma_0(N)^+$ is larger than $\Gamma_0(N)$ and usually called the **Fricke group**.
- When $X_0(N)^+$ has genus zero, there exists a **Hauptmodul** $T_N(q)$ with a $q^{-1}$ pole and rational coefficients.


In [ ]:
#AMS-LaTeX version:
\documentclass[12pt]{amsart}
\usepackage{amsmath, amssymb}

\title{The Meaning of the ``$+$'' in $\Gamma_0(N)^+$}
\author{}
\date{}

\begin{document}

\maketitle

\section*{What does the ``$+$'' in $\Gamma_0(N)^+$ signify?}

The ``$+$'' in the notation $\Gamma_0(N)^+$ indicates an extension of the modular group $\Gamma_0(N)$ by including certain \textbf{Atkin--Lehner involutions}, most notably the \textbf{Fricke involution} $w_N$. This forms a larger group often called the \textbf{Fricke group} at level $N$.

\subsection*{1. The group $\Gamma_0(N)$}

This is the congruence subgroup of $\mathrm{SL}_2(\mathbb{Z})$ defined by
\[
\Gamma_0(N) = \left\{ 
\begin{pmatrix} a & b \\ c & d \end{pmatrix} \in \mathrm{SL}_2(\mathbb{Z}) \;\middle|\; c \equiv 0 \pmod{N}
\right\}.
\]

\subsection*{2. The Fricke involution $w_N$}

The \textbf{Fricke involution} at level $N$ is the matrix
\[
w_N = \begin{pmatrix} 0 & -1/\sqrt{N} \\ \sqrt{N} & 0 \end{pmatrix}.
\]
Although $w_N \notin \mathrm{SL}_2(\mathbb{Z})$, it normalizes $\Gamma_0(N)$ and acts on modular forms via the slash operator. It exchanges the cusps $\infty$ and $0$:
\[
\tau \mapsto -\frac{1}{N\tau}.
\]

\subsection*{3. The group $\Gamma_0(N)^+$}

This group is defined by adjoining $w_N$ to $\Gamma_0(N)$:
\[
\Gamma_0(N)^+ := \langle \Gamma_0(N), w_N \rangle.
\]
It is a finite-index subgroup of $\mathrm{PSL}_2(\mathbb{R})$, and is commonly referred to as the \textbf{Fricke modular group}.

\subsection*{Why is this important?}

\begin{itemize}
  \item The action of $\Gamma_0(N)^+$ identifies more points on the modular curve than $\Gamma_0(N)$ alone.
  \item For many values of $N$, the quotient $X_0(N)^+ = \Gamma_0(N)^+ \backslash \mathbb{H}^*$ has \textbf{genus zero}, even when $X_0(N)$ does not.
  \item In such cases, a \textbf{Hauptmodul} (a distinguished modular function, like $j(\tau)$) exists, usually denoted $T_N(q)$.
\end{itemize}

\subsection*{Example}

The modular function
\[
T_{14}(q) = \frac{1}{q} + 24 + 324q + 3200q^2 + 25650q^3 + \cdots
\]
is the Hauptmodul for $\Gamma_0(14)^+$. It is a modular function of weight $0$, with a simple pole at $\infty$ and rational Fourier coefficients.

\subsection*{Summary}

\begin{itemize}
  \item The ``$+$'' in $\Gamma_0(N)^+$ indicates the inclusion of Atkin--Lehner involutions, especially the Fricke involution.
  \item The resulting group $\Gamma_0(N)^+$ is typically called the \textbf{Fricke group}.
  \item When $X_0(N)^+$ has genus zero, there exists a Hauptmodul $T_N(q)$ with a $q^{-1}$ pole and rational coefficients.
\end{itemize}

\end{document}


In [ ]:
# check Getu theorem
Rx.<x> = PolynomialRing(QQ)
from collections import deque
def c1_operator(lst):
    return [(k + 1) * lst[k] for k in range(len(lst))]

def push_down(j, N, lst):
    d = [1] + lst
    for _ in range(2, j + 1):
        d = [0] + d
    return d[:N]
A_list_s=[]
for N in [2..5]:
    A_ls=[n for n in [0..N]]
    A_list_s+=[A_ls]


        


# MAIN
tests=[]
charpolys=[]
for N in range(1,101):
    lst = [T(j) for j in range(1, N + 1)]

# Build the matrix columns
    column_list = [vector(ZZ, c1_operator(lst))]
    for j in range(1, N):
        pushed = push_down(j, N, lst)
        column_list.append(vector(ZZ, pushed))

# Construct the Sage integer matrix
    mat = Matrix(ZZ, column_list).transpose()

# Compute exact determinant
    determinant = mat.det()
    p = mat.charpoly(x)#.change_ring(SR)
    charpolys+=[(N,p)]
    expected = 24 * sigma(N) * (-1)^N
    test = determinant - expected
    tests+=[test]

# Output
print(tests)

wfile = open('/Users/barrybrent/data/run6aug25no1.txt','wb') 
t = pickle.dumps(str(charpolys))
wfile.write(t)
wfile.close() 

In [1]:
A_list_s=[]
for N in [2..5]:
    A_ls=[n for n in [0..N]]
    A_list_s+=[A_ls]
for ls in A_list_s:
    print(ls)

[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 5]


In [2]:
C_list_s=[]
for N in [2..5]:
    C_ls=[n^2 for n in [0..N]]
    C_list_s+=[C_ls]
for ls in C_list_s:
    print(ls)

[0, 1, 4]
[0, 1, 4, 9]
[0, 1, 4, 9, 16]
[0, 1, 4, 9, 16, 25]


In [3]:
sum?

Signature:      sum(expression, *args, **kwds)
Docstring:     
   Return the symbolic sum \sum_{v = a}^b expression with respect to
   the variable v with endpoints a and b.

   INPUT:

   * "expression" -- a symbolic expression

   * "v" -- a variable or variable name

   * "a" -- lower endpoint of the sum

   * "b" -- upper endpoint of the sum

   * "algorithm" -- (default: "'maxima'")  one of

     * "'maxima'" -- use Maxima (the default)

     * "'maple'" -- (optional) use Maple

     * "'mathematica'" -- (optional) use Mathematica

     * "'giac'" -- (optional) use Giac

     * "'sympy'" -- use SymPy

   EXAMPLES:

      sage: k, n = var('k,n')
      sage: sum(k, k, 1, n).factor()
      1/2*(n + 1)*n

      sage: sum(1/k^4, k, 1, oo)
      1/90*pi^4

      sage: sum(1/k^5, k, 1, oo)
      zeta(5)


     This function only works with symbolic expressions. To sum any
     other objects like list elements or function return values,
     please use python summation, see
     http://do

In [12]:
Rx.<x> = PolynomialRing(QQ)
from collections import deque
import pickle

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]

Rx= PolynomialRing(QQ, 'x')
Pm = PolynomialRing(QQ, 'm')
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def rdr(p,n):
    return QQ.valuation(p)(n)
def sig(j):
    return 24*sigma(j)
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)

def S(n):
    return 24 * (-1)^n * sigma(n)

def T(n):
    return ZZ(ramanujanTau(n+1))

def c1_operator(lst):
    return [(k + 1) * lst[k] for k in range(len(lst))]

def push_down(j, N, lst):
    d = [1] + lst
    for _ in range(2, j + 1):
        d = [0] + d
    return d[:N]

# MAIN
tests=[]
charpolys=[]
for N in range(1,100):
    lst = [T(j) for j in range(1, N + 1)]

# Build the matrix columns
    column_list = [vector(ZZ, c1_operator(lst))]
    for j in range(1, N):
        pushed = push_down(j, N, lst)
        column_list.append(vector(ZZ, pushed))

# Construct the Sage integer matrix
    mat = Matrix(ZZ, column_list).transpose()
   

# Compute exact determinant
    determinant = mat.det()
    p = mat.charpoly(x)#.change_ring(SR)
    charpolys+=[(N,p)]
    epc=extendedPolynomialCoefficient(0,p)
    expected = 24 * sigma(N) 
    test = epc-expected
    tests+=[test]

# Output
print("tests:")
print(tests)

#wfile = open('/Users/barrybrent/data/run6aug25no1.txt','wb') 
#t = pickle.dumps(str(charpolys))
#wfile.write(t)
#wfile.close() 

tests:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
max_val = 18

# Define power series ring with enough precision
R.<x> = PowerSeriesRing(QQ, default_prec = max_val + 2)

# Define f(x)
f = (1/x) * prod((1 - x^k)^(-24) for k in range(1, max_val + 1))

# Series of f(x) - 1/x, truncated to degree max_val - 1
series_exp = (f - 1/x).truncate(max_val)

# Extract coefficients (degree 0 up to max_val - 1)
coeff_list = [series_exp[i] for i in range(series_exp.degree() + 1)]

# Prepend 1
result = [1] + coeff_list

print(result)
#Translated Mathematica code for A006922

[1, 24, 324, 3200, 25650, 176256, 1073720, 5930496, 30178575, 143184000, 639249300, 2705114880, 10914317934, 42189811200, 156883829400, 563116739584, 1956790259235, 6599620022400, 21651325216200]


In [49]:
max_val = 101

# Define power series ring with enough precision
R.<x> = PowerSeriesRing(QQ, default_prec = max_val + 2)

# Define f(x)
f = (1/x) * prod((1 - x^k)^(-24) for k in range(1, max_val + 1))

# Series of f(x) - 1/x, truncated to degree max_val - 1
series_exp = (f - 1/x).truncate(max_val)

# Extract coefficients (degree 0 up to max_val - 1)
coeff_list = [series_exp[i] for i in range(series_exp.degree() + 1)]

# Prepend 1
result = [1] + [coeff_list]
result=flatten(result)

#print(result)
#Translated Mathematica code for A006922

Rx= PolynomialRing(QQ, 'x')
Pm = PolynomialRing(QQ, 'm')
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def rdr(p,n):
    return QQ.valuation(p)(n)
def sig(j):
    return 24*sigma(j)
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)

def S(n):
    return 24 * (-1)^n * sigma(n)

def T(n):
    return ZZ(ramanujanTau(n+1))

def c1_operator(lst):
    return [(k + 1) * lst[k] for k in range(len(lst))]

def push_down(j, N, lst):
    d = [1] + lst
    for _ in range(2, j + 1):
        d = [0] + d
    return d[:N]

# MAIN
tests=[]
charpolys=[]
cffs=[]
ratios=[]
ub=10
for N in range(1,ub):
    lst = [T(j) for j in range(1, N + 1)]

# Build the matrix columns
    column_list = [vector(ZZ, c1_operator(lst))]
    for j in range(1, N):
        pushed = push_down(j, N, lst)
        column_list.append(vector(ZZ, pushed))

# Construct the Sage integer matrix
    mat = Matrix(ZZ, column_list).transpose()
   

# Compute exact determinant
    determinant = mat.det()
    p = mat.charpoly(x)#.change_ring(SR)
    charpolys+=[(N,p)]
    epc=extendedPolynomialCoefficient(1,p)
    cffs+=[epc]
    #expected = result[N-1]
    #ratio=epc/result[n]
    #ratios+=[ratio]
   

# Output
print("cffs:")
print(cffs)
print("result:")
ls=[
        result[n]*(n+1) for n in [0..min(ub,len(cffs)-1)]
    ]
print(ls)
print("ratios:")
ratios+=[cffs[n]/ls[n] for n in [0..min(ub,len(cffs)-1)]]
print(ratios)



#wfile = open('/Users/barrybrent/data/run6aug25no1.txt','wb') 
#t = pickle.dumps(str(charpolys))
#wfile.write(t)
#wfile.close()

cffs:
[1, 48, 972, 12800, 128250, 1057536, 7516040, 47443968, 271607175]
result:
[1, 48, 972, 12800, 128250, 1057536, 7516040, 47443968, 271607175]
ratios:
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [52]:
max_val = 101

# Define power series ring with enough precision
R.<x> = PowerSeriesRing(QQ, default_prec = max_val + 2)

# Define f(x)
f = (1/x) * prod((1 - x^k)^(-24) for k in range(1, max_val + 1))

# Series of f(x) - 1/x, truncated to degree max_val - 1
series_exp = (f - 1/x).truncate(max_val)

# Extract coefficients (degree 0 up to max_val - 1)
coeff_list = [series_exp[i] for i in range(series_exp.degree() + 1)]

# Prepend 1
result = [1] + [coeff_list]
result=flatten(result)

#print(result)
#Translated Mathematica code for A006922

Rx= PolynomialRing(QQ, 'x')
Pm = PolynomialRing(QQ, 'm')
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def rdr(p,n):
    return QQ.valuation(p)(n)
def sig(j):
    return 24*sigma(j)
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)

def S(n):
    return 24 * (-1)^n * sigma(n)

def T(n):
    return ZZ(ramanujanTau(n+1))

def c1_operator(lst):
    return [(k + 1) * lst[k] for k in range(len(lst))]

def push_down(j, N, lst):
    d = [1] + lst
    for _ in range(2, j + 1):
        d = [0] + d
    return d[:N]

# MAIN
tests=[]
charpolys=[]
cffs=[]
ratios=[]
ub=100
for N in range(1,ub):
    lst = [T(j) for j in range(1, N + 1)]

# Build the matrix columns
    column_list = [vector(ZZ, c1_operator(lst))]
    for j in range(1, N):
        pushed = push_down(j, N, lst)
        column_list.append(vector(ZZ, pushed))

# Construct the Sage integer matrix
    mat = Matrix(ZZ, column_list).transpose()
   

# Compute exact determinant
    determinant = mat.det()
    p = mat.charpoly(x)#.change_ring(SR)
    charpolys+=[(N,p)]
    epc=extendedPolynomialCoefficient(1,p)
    cffs+=[epc]
    #expected = result[N-1]
    #ratio=epc/result[n]
    #ratios+=[ratio]
   

# Output

ls=[
        result[n]*(n+1) for n in [0..min(ub,len(cffs)-1)]
    ]
print("result:")
print(result)
print("ratios:")
ratios+=[cffs[n]/ls[n] for n in [0..min(ub,len(cffs)-1)]]
print(ratios)



#wfile = open('/Users/barrybrent/data/run6aug25no1.txt','wb') 
#t = pickle.dumps(str(charpolys))
#wfile.write(t)
#wfile.close()

result:
[1, 24, 324, 3200, 25650, 176256, 1073720, 5930496, 30178575, 143184000, 639249300, 2705114880, 10914317934, 42189811200, 156883829400, 563116739584, 1956790259235, 6599620022400, 21651325216200, 69228721526400, 216108718571250, 659641645039360, 1971466420726656, 5776331152550400, 16610409114771900, 46925988716146176, 130362155499200220, 356418628326241024, 959788304511313500, 2547447689037081600, 6668597583531616856, 17227666361525437440, 43946595512833354821, 110753578062185091200, 275889636433651636800, 679603117953171550464, 1656159528253893300680, 3994373142513720019584, 9537992210458653910200, 22556911735643814336000, 52851854116498243371768, 122725297901736598060800, 282506903283485314589800, 644860824976888592486400, 1460021679052070827818150, 3279574124861933907622400, 7310437360199294416934040, 16174647177339633952121856, 35529150168048315816004075, 77496468553699048793894400, 167884450803343339733543652, 361285884535944808280618880, 772467683080630774831410348, 16412